## Adding routes to neighborhood -- run after routes.geojson has been generated.

Then use `geojson-shave` CLI tool to minify.

In [ ]:
import geopandas as gpd
import json

# Load neighborhoods and routes GeoJSON files
neighborhoods_gdf = gpd.read_file("nyc-neighborhoods.geojson")
routes_gdf = gpd.read_file("routes.geojson")

# Ensure geometries are in the correct coordinate reference system (CRS)
neighborhoods_gdf = neighborhoods_gdf.to_crs(epsg=4326)
routes_gdf = routes_gdf.to_crs(epsg=4326)

# Function to find neighborhoods a route passes through
def get_neighborhoods_for_route(route_geometry):
    matched_neighborhoods = []
    
    for _, neigh in neighborhoods_gdf.iterrows():
        if route_geometry.intersects(neigh.geometry):
            matched_neighborhoods.append({
                "n": neigh["neighborhood"],
                "b": neigh["borough"]
            })
    
    return matched_neighborhoods

# Process each route and add neighborhood data
routes_gdf["neighborhoods"] = routes_gdf["geometry"].apply(get_neighborhoods_for_route)

# Convert back to a GeoJSON format
routes_geojson = json.loads(routes_gdf.to_json())

# Save updated GeoJSON file
with open("routes_with_neighborhoods.geojson", "w") as f:
    json.dump(routes_geojson, f, indent=2)

print("Updated GeoJSON file saved as 'routes_with_neighborhoods.geojson'")
